In [159]:
import matplotlib.pyplot as plt, pandas as pd,numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

In [160]:
#reading testing and training files using pandas and dropping columns with least covariance with systolic and diastolic bp
df = pd.read_csv("ML101_train_dataset.csv")
df_test=pd.read_csv("ML101_dataset_test_feature.csv")
df.dropna(subset=['Caloric Intake','Height','Cholesterol level','Average Daily Steps','Hours of Sleep','Gender','LifeStyle','Systolic BP','Diastolic BP'], inplace=True)
df=df.reset_index(drop=True)
df_test = df_test.drop('Gender', axis=1)
df = df.drop('Gender', axis=1)
df_test = df_test.drop('Average Daily Steps', axis=1)
df = df.drop('Average Daily Steps', axis=1)
df_test = df_test.drop('Hours of Sleep', axis=1)
df = df.drop('Hours of Sleep', axis=1)
attributes = df.columns

In [161]:
#replacing nan values of columns using mean  
columns_to_impute = ['Caloric Intake','Age','Weight','Height','Cholesterol level','Blood Sugar level']
imputer = SimpleImputer(strategy='mean')
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

In [162]:
#creating another numpy arrays such that each row starts with constant value 1 
y1=df['Systolic BP']
y2=df['Diastolic BP']
y3=df['LifeStyle']
x_test=np.array(df_test)
x=np.array(df[attributes[0:len(attributes)-3]])
x_test1=np.array([[1]*(len(x[0])+1)]*len(x_test),dtype=float)
x_final=np.array([[1]*(len(x[0])+1)]*len(x),dtype=float)
for i in range(1,len(x_final[0])):
    for j in range(len(x)):
        x_final[j][i]=x[j][i-1]
for i in range(1,len(x_test1[0])):
    for j in range(len(x_test1)):
        x_test1[j][i]=x_test[j][i-1]

In [163]:
#finding parameters for both systolic and diastolic bp by calculating dagger of data matrix and then multiplying it with target variable and then
#writing predicted values to another csv file and also adding two columns in test data dataframe so as to use during classification 
systolic_bp=[]
diastolic_bp=[]
w_sys=np.matmul(np.matmul(np.linalg.inv(np.matmul(x_final.transpose(),x_final)),x_final.transpose()),y1)
w_dia=np.matmul(np.matmul(np.linalg.inv(np.matmul(x_final.transpose(),x_final)),x_final.transpose()),y2)
for i in range(len(x_test1)):
    systolic_bp.append(np.matmul(w_sys.transpose(),x_test1[i]))
    diastolic_bp.append(np.matmul(w_dia.transpose(),x_test1[i]))
systolic_bp=np.array(systolic_bp)
diastolic_bp=np.array(diastolic_bp)
op_df=pd.DataFrame({"Systolic BP":pd.Series(systolic_bp),"Diastolic BP":pd.Series(diastolic_bp)})
op_df=op_df.rename_axis("ID")            

op_df.to_csv("regression_cipher.csv")

df_test['Systolic BP']=systolic_bp
df_test['Diastolic BP']=diastolic_bp
x_test2=np.array(df_test)
x_train2=np.array(df[attributes[:len(attributes)-1]])

In [164]:
#creating an adaboost classifier with the decision tree as the base classifier and fitting the ensemble classifier to data and making predictions and 
#writing it to csv
base_classifier = DecisionTreeClassifier()
ensemble_classifier = AdaBoostClassifier(base_classifier, n_estimators=50)
ensemble_classifier.fit(x_train2, y3)
y_pred = ensemble_classifier.predict(x_test2)
df_final=pd.DataFrame({"LifeStyle":y_pred})
df_final.to_csv("classification_cipher.csv")